## Introduction

The work from [Sathe et al. 2015](https://doi.org/10.5194/amt-8-729-2015) introduces an alternative scanning strategy to be used by wind Doppler lidars for estimating turbulence. This new strategy consists of measuring the radial wind on five equally spaced azimuths with fixed elevation, different from 0 or 90 degrees,  and one additional measurement from the zenith. From those observations, the Reynolds stress tensor elements can be calculated following the method suggested developed by Sathe et al. 2015. 

This example shows how to use the lidarwind package to calculate the Reynolds stress tensor elements from the six beam observations obtained by the WindCube lidar 200s. This example is structured according to the steps listed below.

## Steps:

0) Download data from zenodo
1) Read the WindCube's data
2) Merge data from one hour of observations
3) Re-structure the data for using the wind retrieval
4) Calculate the Reynolds stress tensor elements
5) Visualisation

In [ ]:
import glob
import gdown

import matplotlib.pyplot as plt
import lidarwind
from lidarwind.utilities import sample_data

### Step 0: Downloading sample data from zenodo

Using pooch the sample dataset will be cached in your system.

In [ ]:
file_list = sample_data('wc_6beam')

### Step 1 and 2: reading and merging the 6-beam data

In [ ]:
file_list = sorted(file_list)
merged_ds = lidarwind.DataOperations(file_list).merged_data

Below you can see all the variables available on the original WindCube's data. Not all of them are needed for retrieving the wind profiles. Note that the variables from the vertical observations are kept separate from the slanted observations. 

In [ ]:
merged_ds

### Step 3: re-structuring the data

The re-structured object contains all the information needed to calculate the Reynolds stress tensor elements.

In [ ]:
restruct_data = lidarwind.GetRestructuredData(merged_ds)

### Step 4: Calculate the Reynolds stress tensor elements

To calculate Reynolds stress tensor elements, you need to define a time window (a frequency). Since lidarwind takes advantage of the xarray library, the frequency for calculating those elements is defined in terms of the number of profiles. However, a frequency in terms of time is more practical. In the following fuor lines, a time frequency will be related to its equivalent in terms of profile frequency.

In [ ]:
# desired time windown in minutes
time_window = 5

# duration of one minute in seconds
minute_lenght = 60

# vertical observations time resolution in seconds
time_resolution = restruct_data.data_transf_90.time.diff(dim='time').values * 1e-9
time_resolution = int(time_resolution[0])

# frequency convertion from minutes to profile number
freq = (minute_lenght/time_resolution)*time_window
freq = int(freq)

Applying the 6-beam method to calculate Reynolds stress tensor elements

In [ ]:
turb_data = lidarwind.SixBeamMethod(restruct_data, freq=freq, freq90=freq)

Have a look at the Reynolds elements dataset

In [ ]:
turb_data.var_comp_ds

Below we will have the first impressions of the retrieved data. Note that there are negative variances, which is mathematically impossible. Those negative is a known artefact of the 6-beam method.

### Step 5: Visualisation

In [ ]:
turb_data.var_comp_ds.var_u.plot(x='time', cmap='Spectral', vmin=-5, vmax=5)
plt.show()

turb_data.var_comp_ds.var_v.plot(x='time', cmap='Spectral', vmin=-5, vmax=5)
plt.show()

turb_data.var_comp_ds.var_w.plot(x='time', cmap='Spectral', vmin=0, vmax=1)
plt.show()